# Excel 的内建功能：錄製巨集

在開始之前講一下這個實用的 Excel 功能，Excel 會把一連串使用者的操作都錄製下來

![](https://www.dropbox.com/s/6h4xzhgfc6a7k1r/be1200dc-fdf2-48d0-a8e6-b23a1a885339.png?dl=1)

實際上它的運作原理是把使用者的操作轉成 VBA 的子程序

這個功能看似强大，但其實有很多局限

想理解更多的同學可以看一下 [Microsoft 的官方文件](https://support.office.com/zh-tw/article/%E4%BD%BF%E7%94%A8%E5%B7%A8%E9%9B%86%E9%8C%84%E8%A3%BD%E5%99%A8%E8%87%AA%E5%8B%95%E5%8C%96%E5%B7%A5%E4%BD%9C-974ef220-f716-4e01-b015-3ea70e64937b)

# VBA 語言簡介

用一句話形容用 VBA 為 Excel 寫自動化的程式，就是**針對一個或多個儲存格進行操作**

針對一個或多個儲存做修改時，我們勢必需要先選擇我們想要操作的儲存格，也因此，在 VBA 最常被用到的兩個函數分別是：

### Cells() 函數：

用法：**Cells(row值, column值)**

今天當我想賦值給 A1 儲存格時：
```vb
Cells(1, 1).value = 123
```
問題是這樣的寫法非常不直覺，所以 VBA 也支援以下寫法：
```vb
Cells(1, "A").value = 123
```

### Range() 函數：

當我需要針對一個或多個儲存格(一個 Excel 試算表的範圍)去進行操作時：

用法：**Range("起點:結束點")**

將 A1 儲存格的值設定成 123
```vb
Range("A1").value = 123
```
將範圍 A1 到 C3 共九個儲存格的值設定成 123

```vb
Range("A1:C3").value = 123
```

當然，字串在用 loop 執行時會增加程式碼的複雜度(字串拼接)
所以 Range 有另一種用法：**Range(Cells(起點), Cells(結束點))**

```vb
Range(Cells(1, "A"), Cells(3, "C")).value = 123
```


### Dot Notation (句點表示法)：

大部分的物件導向程式語言都支援的語法：

**<物件名稱>.<屬性名稱> = <被賦予的值>**

# Python 與 Excel 整合的兩大主流套件

Python 與 Excel 整合主要的兩個套件就是
- **openpyxl**
- **xlwings**

### openpyxl vs xlwings

- 若你只想單純的匯出 Excel 報表，openpyxl 就足夠使用
- 若你希望能夠實作出類似 VBA 的所見即所得 (WYSIWYG) 的功能，xlwings 比較適合你


# 先 import 我們所需要的模組

In [ ]:
import openpyxl
import os

In [ ]:
# 切換工作目錄
os.chdir(r"stock_price_data.xlsx 所在的路徑")
# 將 stock_price_data.xlsx 的内容存入 workbook 變數
workbook = openpyxl.load_workbook("stock_price_data.xlsx")
# 將 stock_price_data.xlsx 名爲 2330 的試算表存入 sheet 變數
sheet = workbook["2330"]
# 用 cell 函數截取 row =1, column = 2 的儲存格的值
result = sheet.cell(row=1, column=2).value
print(result)

# 計算第二天的日報酬率

In [ ]:
# 取得 B4 儲存格(第三天)的資料
today_price = sheet.cell(row=4, column=2).value
today_price

In [ ]:
# 取得 B3 儲存格(第二天)的資料
yesterday_price = sheet.cell(row=3, column=2).value
yesterday_price

In [ ]:
# 計算報酬率
return_value = (today_price - yesterday_price) / yesterday_price

In [ ]:
# 最後需要把值寫入儲存格 C4 (別忘了關掉 Excel 檔案)
sheet.cell(row=4, column=3).value = return_value

# 別忘了要存檔
# workbook.save("stock_price_data.xlsx")

# 隨堂練習：計算出每一天的報酬率

In [ ]:
for i in range(3, 97):
    # 算出每一個 row 的報酬率
    today_price = _______________________
    yesterday_price = _______________________
    daily_return = _______________________
    # 寫入每一個儲存格
    _______________________
    # 印出該行報酬率
    print(daily_return)

# 算出所有儲存格後，存檔
workbook.save("stock_price_data.xlsx")

# openpyxl 小結

- 功能强大，語法貼近 Python
- 若是有功能需要做到寫入儲存格，就需要關掉 Excel 才可以運作 (無法所見及所得，開發流程與效率相對差)
- 若是需要寫入值，需要在程式的最後一行加上 workbook.save() (相對麻煩)
- 適用於大型系統輸出 Excel 報表的功能

# 接下來我們來探索 xlwings 套件

In [12]:
import xlwings as xw

In [13]:
# 打開你的 stock_price_data.xlsx 檔案
wb = xw.Book(r'C:\Users\user\Desktop\305_EXCEL_PYTHON\Lesson1\stock_price_data.xlsx')

### xlwings 的 cells() 與 range()

在語法上非常貼近 VBA 的 cells() 與 range():

```python
cells(1, 'A').value = 123

range("A1:C3").value = 123
```

這樣的好處是大幅降低了 VBA 開發者切換至 Python 的學習成本


## 使用 cells() 的小叮嚀

今天我若想擷取 B1 儲存格的值，可以這樣寫：
```python
tsmc_sheet.cells(1, 'B').value
```

### cells(儲存格的row值，儲存格的column值)

儲存格的column值可以用字串 'B' 或 數字 2 表示

### 但是！

**若你是用 Mac, xlwings 不支援用字串表示 column 值**

這是 xlwings 在 Mac 上的極限，所以若你是用 Mac，上述程式碼需改寫成：

```python
tsmc_sheet.cells(1, 2).value
```

In [14]:
# 從該 Excel 檔案裡找出名為 2330 的試算表，存入 tsmc_sheet 這個變數裏
tsmc_sheet = wb.sheets['2330']
tsmc_sheet

<Sheet [stock_price_data.xlsx]2330>

In [15]:
tsmc_sheet.cells(1, "F").value = 123


tsmc_sheet.range("F1:F3").value = 123

# 計算報酬率

In [16]:
# 算出所有的報酬率
for i in range(3, 97):
    daily_return = (tsmc_sheet.cells(i, 2).value - tsmc_sheet.cells(i-1, 2).value) / tsmc_sheet.cells(i-1, 2).value
    tsmc_sheet.cells(i, 3).value = daily_return

問題來了，要是試算表上**多了或少了一筆資料，原本的程式就爆炸了...**

**若你寫出了一個需要頻繁修改的程式，那就失去了自動化的意義**

我們需要一個能夠動態偵測我們有幾筆資料需要計算的方法

# 動態偵測資料筆數的方法

在 VBA 的世界裡，有個好用的函數叫做 End()，可以動態偵測一堆**連續範圍的最後一個儲存格**
```vb
last_row = Range("A1").End(xlDown).Row
```
會從 A1 儲存格開始，**往下**查找到最後一個有值的儲存格
```vb
last_col = Range("A1").End(xlToRight).Column
```
會從 A1 儲存格開始，**往右**查找到最後一個有值的儲存格

In [17]:
# 這邊就來介紹一下 xlwings 的 end 功能
from xlwings.constants import Direction


# 會 VBA 的人，馬上可以看出 xlwings 原汁原味的實作了 Python 版的 End() 函數
# 從 A1 儲存格開始，往下查找到最後一個有值的儲存格
last_cell = tsmc_sheet.range('A1').end(Direction.xlDown)
# 把從 A1 開始，往下最後一個有值的儲存格底色設定成紅色
last_cell.color = (255, 0, 0)

In [20]:
from xlwings.constants import Direction
# 查找從 A1 開始、有值的範圍的最右下角的儲存格，將該儲存格存入 last_cell 變數，將其底色標為紅色
last_cell = tsmc_sheet.range('A1').end(Direction.xlDown).end(Direction.xlToRight)

# 使用 .row 屬性取得 row 值，也就是我們的資料有幾行
last_row = last_cell.row

# 隨堂練習

利用動態偵測最後一個 row 的方式計算每一天的報酬率

```python
num_of_rows = tsmc_sheet.range('B1').end(Direction.xlDown).row

for i in range(3, _________):
    daily_return = _________________________________
    ______________________ = daily_return
```

# 回家作業

1. Cells Examples 所有的題目
2. 計算所有的三日移動平均
3. 計算出所有資料的五日加權移動平均 (加分題)

In [21]:
# 算出所有的三日移動平均
for i in range(4, last_row + 1):
    today_price = tsmc_sheet.cells(i, 2).value
    yesterday_price = tsmc_sheet.cells(i-1, 2).value
    day_b4_yesterday =tsmc_sheet.cells(i-2, 2).value
    
    moving_average = (today_price + yesterday_price + day_b4_yesterday) / 3
    tsmc_sheet.cells(i,  4).value = moving_average

# 相關資源：

Medium: 

[當大蟒蛇語言遇上老牌試算表軟體](https://medium.com/pyradise/%E7%95%B6%E5%A4%A7%E8%9F%92%E8%9B%87%E8%AA%9E%E8%A8%80%E9%81%87%E4%B8%8A%E8%80%81%E7%89%8C%E8%A9%A6%E7%AE%97%E8%A1%A8%E8%BB%9F%E9%AB%94-fc7947792000)

[當大蟒蛇語言遇上老牌試算表軟體(2)](https://medium.com/pyradise/%E7%95%B6%E5%A4%A7%E8%9F%92%E8%9B%87%E8%AA%9E%E8%A8%80%E9%81%87%E4%B8%8A%E8%80%81%E7%89%8C%E8%A9%A6%E7%AE%97%E8%A1%A8%E8%BB%9F%E9%AB%94-2-2e1e825b31fe)

xlwings 官網：

[官網連結](https://www.xlwings.org/)